In [ ]:
%matplotlib inline
from matplotlib import pylab as plt
from netCDF4 import Dataset
import rasterio
from geonotebook.wrappers import RasterData

In [ ]:
nc = Dataset('tasmax.nc')
tasmax = nc.variables['tasmax']
plt.imshow(np.flipud(tasmax[0, ...]))

In [ ]:
N, height, width = tasmax.shape

lat = nc.variables['lat']
lon = nc.variables['lon']
x0, y0 = lon[width / 2] - 360, lat[-1]
dx, dy = lon[1] - lon[0], lat[1] - lat[0]

crs = {'init': 'EPSG:4326'}
transform = rasterio.transform.from_origin(x0, y0, dx, dy)

In [ ]:
with rasterio.open(
    'tasmax.tiff', 'w', driver='GTiff',
    height=height, width=width,
    count=1, dtype=tasmax.dtype,
    crs=crs, transform=transform,
    nodata=tasmax.missing_value) as tiff:

    for i in xrange(1):
        data = np.flipud(
            np.roll(
                tasmax[i, ...],
                width / 2,
                axis=1
            )
        )
        tiff.write(data, i + 1)
    tiff.write_mask(True)

In [ ]:
for layer in M.layers:
    M.remove_layer(layer)
    
rd = RasterData('tasmax.tiff')
cmap = plt.get_cmap('viridis', 16)
M.add_layer(rd[1], opacity=0.75, colormap=cmap)